<a href="https://colab.research.google.com/github/muhammadnouman911/Dataset-Preprocessing/blob/main/Copy_of_Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import pandas as pd

# Upload XLSX file
uploaded = files.upload()

# Convert to CSV
excel_file = list(uploaded.keys())[0]
df = pd.read_excel(excel_file)
csv_file = 'data.csv'
df.to_csv(csv_file, index=False)
print("✅ Excel converted to CSV")


Saving Book1.xlsx to Book1.xlsx
✅ Excel converted to CSV


In [ ]:
!pip install yt-dlp ffmpeg-python tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 81.7 MB/s eta 0:00:00


In [ ]:
import os
import cv2
import pandas as pd
import yt_dlp
import ffmpeg
from tqdm import tqdm
import time
import shutil
from multiprocessing import Pool, cpu_count
import zipfile

# --- Configuration ---
CSV_PATH = "/content/data.csv"
OUTPUT_DIR = "dataset"
AUDIO_DIR = os.path.join(OUTPUT_DIR, "audio")
VIDEO_DIR = os.path.join(OUTPUT_DIR, "video")
FACE_DIR = os.path.join(OUTPUT_DIR, "faces")
FULL_VIDEO_DIR = os.path.join(OUTPUT_DIR, "full_videos")
COOKIES_FILE = "cookies.txt" if os.path.exists("cookies.txt") else None
LOG_FILE = "failed_downloads.log"
MAX_RETRIES = 3  # Increased retries
PROXY = None
WORKERS = max(1, cpu_count() - 1)  # Slightly more workers

# Create directories
os.makedirs(AUDIO_DIR, exist_ok=True)
os.makedirs(VIDEO_DIR, exist_ok=True)
os.makedirs(FULL_VIDEO_DIR, exist_ok=True)
os.makedirs(FACE_DIR, exist_ok=True)

# --- Enhanced Logging ---
def log_failure(youtube_id, reason, details=""):
    with open(LOG_FILE, "a") as f:
        f.write(f"{youtube_id},{reason},{details}\n")

# --- Improved Video Download ---
def download_video(youtube_id, output_path, retry=0):
    ydl_opts = {
        "format": "bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best",
        "outtmpl": output_path,
        "quiet": True,
        "ignoreerrors": True,
        "extractor_args": {"youtube": {"skip": ["hls", "dash"]}},
        "socket_timeout": 60,  # Increased timeout
        "retries": 5,
        "noplaylist": True,
    }

    if COOKIES_FILE:
        ydl_opts["cookiefile"] = COOKIES_FILE
    if PROXY:
        ydl_opts["proxy"] = PROXY

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(
                f"https://www.youtube.com/watch?v={youtube_id}",
                download=True
            )
            if info is None:
                raise Exception("Video info not available")
        return os.path.exists(output_path)
    except Exception as e:
        if retry < MAX_RETRIES:
            time.sleep(2 ** retry)  # Exponential backoff
            return download_video(youtube_id, output_path, retry + 1)
        log_failure(youtube_id, "Download failed", str(e))
        return False

# --- Robust Video Processing ---
def process_video_segment(row):
    try:
        youtube_id, start, end, x, y = row
        base_name = f"{youtube_id}_{start}_{end}"
        audio_path = os.path.join(AUDIO_DIR, f"{base_name}.wav")
        segment_video_path = os.path.join(VIDEO_DIR, f"{base_name}.mp4")
        face_path = os.path.join(FACE_DIR, f"{base_name}.jpg")
        full_video_path = os.path.join(FULL_VIDEO_DIR, f"{youtube_id}_full.mp4")

        # Skip if already processed
        if os.path.exists(audio_path) and os.path.exists(face_path):
            return True

        # Download full video if needed
        if not os.path.exists(full_video_path):
            if not download_video(youtube_id, full_video_path):
                return False

        # Extract segment
        try:
            (
                ffmpeg.input(full_video_path, ss=start, to=end)
                .output(segment_video_path, c="copy")
                .run(quiet=True, overwrite_output=True)
            )
        except ffmpeg.Error as e:
            log_failure(youtube_id, "Segment extraction failed", str(e.stderr))
            return False

        # Extract audio
        try:
            (
                ffmpeg.input(segment_video_path)
                .output(audio_path, ac=1, ar=16000, acodec="pcm_s16le")
                .run(quiet=True, overwrite_output=True)
            )
        except ffmpeg.Error as e:
            log_failure(youtube_id, "Audio extraction failed", str(e.stderr))
            return False

        # Extract single face frame
        try:
            cap = cv2.VideoCapture(segment_video_path)
            if not cap.isOpened():
                raise Exception("Could not open video segment")

            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            middle_frame = max(0, total_frames // 2)
            cap.set(cv2.CAP_PROP_POS_FRAMES, middle_frame)

            ret, frame = cap.read()
            if not ret:
                raise Exception("Could not read middle frame")

            h, w = frame.shape[:2]
            face_x = int(float(x) * w)
            face_y = int(float(y) * h)

            y1, y2 = max(0, face_y - 112), min(h, face_y + 112)
            x1, x2 = max(0, face_x - 112), min(w, face_x + 112)
            face = frame[y1:y2, x1:x2]

            if face.size == 0:
                raise Exception("Empty face crop")

            face = cv2.resize(face, (224, 224))
            cv2.imwrite(face_path, face)
            cap.release()

        except Exception as e:
            log_failure(youtube_id, "Face extraction failed", str(e))
            return False

        return True

    except Exception as e:
        log_failure(row[0], "Processing failed", str(e))
        return False

# --- Main Execution ---
if __name__ == "__main__":
    # Clear previous log
    with open(LOG_FILE, "w") as f:
        f.write("youtube_id,reason,details\n")

    # Load and validate data
    try:
        df = pd.read_csv(CSV_PATH, header=None)
        df.columns = ["youtube_id", "start", "end", "x", "y"]
        df = df.dropna()
        print(f"Loaded {len(df)} valid entries from CSV")
        rows_list = list(df.itertuples(index=False, name=None))
    except Exception as e:
        print(f"Failed to load CSV: {str(e)}")
        exit()

    # Process with enhanced error handling
    print(f"Starting processing with {WORKERS} workers...")
    success = 0
    failure = 0

    with Pool(WORKERS) as pool:
        with tqdm(total=len(rows_list), unit="video") as pbar:
            for result in pool.imap_unordered(process_video_segment, rows_list):
                if result:
                    success += 1
                else:
                    failure += 1
                pbar.update(1)
                pbar.set_postfix({"success": success, "failed": failure})

    # Generate report
    print("\nProcessing complete!")
    print(f"Successfully processed: {success} videos")
    print(f"Failed: {failure} videos")

    if failure > 0:
        print(f"\nFailure reasons summary:")
        failed_df = pd.read_csv(LOG_FILE, header=None, names=["id", "reason", "details"])
        print(failed_df["reason"].value_counts())

    print("\nWould you like to package the dataset? (y/n)")
    if input().strip().lower() == 'y':
        print("Packaging dataset...")
        shutil.make_archive("dataset", 'zip', OUTPUT_DIR)
        print("Dataset packaged to dataset.zip")

Loaded 49 valid entries from CSV
Starting processing with 1 workers...


  0%|          | 0/49 [00:00<?, ?video/s]ERROR: [youtube] u5MPyrRJPmc: The uploader has not made this video available in your country
This video is available in Andorra, United Arab Emirates, Afghanistan, Antigua and Barbuda, Anguilla, Albania, Armenia, Angola, Antarctica, Argentina, American Samoa, Austria, Australia, Aruba, Åland Islands, Azerbaijan, Bosnia and Herzegovina, Barbados, Bangladesh, Belgium, Burkina Faso, Bulgaria, Bahrain, Burundi, Benin, Saint Barthélemy, Bermuda, Brunei Darussalam, Bolivia, Plurinational State of, Bonaire, Sint Eustatius and Saba, Brazil, Bahamas, Bhutan, Bouvet Island, Botswana, Belarus, Belize, Canada, Cocos (Keeling) Islands, Congo, the Democratic Republic of the, Central African Republic, Congo, Switzerland, Côte d'Ivoire, Cook Islands, Chile, Cameroon, China, Colombia, Costa Rica, Cuba, Cape Verde, Curaçao, Christmas Island, Cyprus, Czech Republic, Germany, Djibouti, Denmark, Dominica, Dominican Republic, Algeria, Ecuador, Estonia, Egypt, Western

  4%|▍         | 2/49 [00:36<14:41, 18.75s/video, success=1, failed=1]

  6%|▌         | 3/49 [00:42<10:06, 13.19s/video, success=2, failed=1]ERROR: [youtube] VvcwAGkSy2o: Sign in to confirm your age. This video may be inappropriate for some users. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies
ERROR: [youtube] VvcwAGkSy2o: Sign in to confirm your age. This video may be inappropriate for some users. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies
ERROR: [youtube] VvcwAGkSy2o: Sign in to confirm your age. This video may be inappropria

 10%|█         | 5/49 [01:07<09:09, 12.49s/video, success=3, failed=2]

...

 92%|█████████▏| 45/49 [07:34<00:30,  7.58s/video, success=31, failed=14]

 94%|█████████▍| 46/49 [07:39<00:21,  7.10s/video, success=32, failed=14]

 96%|█████████▌| 47/49 [07:45<00:13,  6.60s/video, success=32, failed=15]ERROR: [youtube] HMGg-84WPAQ: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies
ERROR: [youtube] HMGg-84WPAQ: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies
ERROR: [youtube] HMGg-84WPAQ: Private video. Sign in if you've been granted access to this video. Use -

100%|██████████| 49/49 [08:06<00:00,  9.93s/video, success=33, failed=16]



Processing complete!
Successfully processed: 33 videos
Failed: 16 videos

Failure reasons summary:
reason
Download failed           10
Face extraction failed     6
reason                     1
Name: count, dtype: int64

Would you like to package the dataset? (y/n)
y
Packaging dataset...
Dataset packaged to dataset.zip


In [ ]:
from google.colab import files
import os

# Path to your dataset zip file (same as created in previous step)
ZIP_PATH = "dataset.zip"

def download_dataset():
    # Verify the zip file exists
    if not os.path.exists(ZIP_PATH):
        raise FileNotFoundError(f"Dataset zip file not found at {ZIP_PATH}")

    # For Google Colab
    try:
        from google.colab import files
        print("Downloading dataset from Google Colab...")
        files.download(ZIP_PATH)
        return True
    except:
        pass

    # For local Python environment
    try:
        import webbrowser
        import urllib.parse
        print("Opening dataset in browser for download...")
        webbrowser.open(f"file://{urllib.parse.quote(os.path.abspath(ZIP_PATH))}")
        return True
    except:
        pass

    # Fallback - just print the path
    print(f"Couldn't initiate automatic download. Please manually download from: {os.path.abspath(ZIP_PATH)}")
    return False

# Add this at the end of your main script (after packaging)
if __name__ == "__main__":
    # ... [your existing code] ...

    # After packaging isa complete
    print("\nWould you like to download the dataset now? (y/n)")
    response = input().strip().lower()
    if response == 'y':
        if download_dataset():
            print("Download initiated successfully!")
        else:
            print("Could not initiate automatic download. Please check the file path.")


Would you like to download the dataset now? (y/n)
y


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Download initiated successfully!
